In [2]:
import pandas as pd
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [3]:
file_name = "learning_rate_weight_decay_results.xlsx"

In [4]:
data = pd.read_excel(file_name, index_col=0)
data = data.round(4)

In [5]:
data.sort_values(by="Average of Metrics", inplace=True, ascending=False)
data.head(5)

,Model,Learning Rate,Weight Decay,Average of Metrics,Expectation MSE
3,encoder_with_MLP,0.001,0.100,0.8197,0.0226
4,encoder_with_MLP,0.001,0.010,0.8183,0.0222
5,encoder_with_MLP,0.001,0.001,0.8181,0.0221
0,encoder_with_MLP,0.010,0.100,0.8121,0.0216
12,rnn_with_MLP,0.001,0.100,0.8108,0.0211


In [6]:
mean_metric = round(data["Average of Metrics"].mean(), 4)
std_metric = round(data["Average of Metrics"].std(), 4)
print(f"Average of Metrics Mean: {mean_metric} +/- {std_metric}")

Average of Metrics Mean: 0.7442 +/- 0.1464


In [7]:
mean_expectation_mse = round(data["Expectation MSE"].mean(), 4)
std_expectation_mse = round(data["Expectation MSE"].std(), 4)
print(f"Expectation MSE Mean: {mean_expectation_mse} +/- {std_expectation_mse}")

Expectation MSE Mean: 0.0427 +/- 0.0471


In [8]:
def compute_mean_std_table(data: pd.DataFrame, column: str) -> pd.DataFrame:
    return round(
        data.groupby(column)
        .agg(
            {
                "Average of Metrics": ["mean", "std"],
                "Expectation MSE": ["mean", "std"],
            }
        )
        .rename_axis(column),
        4,
    )


def compute_f_statistic_p_value(
    data: pd.DataFrame, column: str
) -> pd.DataFrame:
    grouped_data = data.groupby(column)

    group_values = [
        group["Average of Metrics"].values for _, group in grouped_data
    ]

    f_statistic, p_value = f_oneway(*group_values)

    return round(f_statistic, 4), round(p_value, 4)

In [9]:
display(compute_mean_std_table(data, "Model"))

f_statistic, p_value = compute_f_statistic_p_value(data, "Model")

print(f"F-statistic: {f_statistic}, P-value: {p_value}") 


Average of Metrics         Expectation MSE        
                               mean     std            mean     std
Model                                                              
encoder_with_MLP             0.7398  0.1095          0.0522  0.0455
rnn_with_MLP                 0.8069  0.0022          0.0212  0.0004
simple_ANN                   0.6860  0.2222          0.0548  0.0660

F-statistic: 1.6142, P-value: 0.2199


In [10]:
display(compute_mean_std_table(data, "Learning Rate"))

f_statistic, p_value = compute_f_statistic_p_value(data, "Learning Rate")

print(f"F-statistic: {f_statistic}, P-value: {p_value}") 


Average of Metrics         Expectation MSE        
                            mean     std            mean     std
Learning Rate                                                   
0.0001                    0.7323  0.1039          0.0520  0.0456
0.0010                    0.8085  0.0094          0.0216  0.0005
0.0100                    0.6919  0.2256          0.0546  0.0661

F-statistic: 1.5318, P-value: 0.2365


In [11]:
display(compute_mean_std_table(data, "Weight Decay"))

f_statistic, p_value = compute_f_statistic_p_value(data, "Weight Decay")

print(f"F-statistic: {f_statistic}, P-value: {p_value}") 

Average of Metrics         Expectation MSE        
                           mean     std            mean     std
Weight Decay                                                   
0.001                    0.7247  0.1760          0.0482  0.0550
0.010                    0.7253  0.1761          0.0482  0.0551
0.100                    0.7826  0.0721          0.0318  0.0308

F-statistic: 0.4436, P-value: 0.6469


In [13]:
data = data.drop(columns=["Expectation MSE"])

# Group the data by the model and compute the correlation for each group
correlations = data.groupby("Model").apply(
    lambda x: x[["Learning Rate", "Weight Decay", "Average of Metrics"]].corr()
)

correlations

Learning Rate  Weight Decay  \
Model                                                              
encoder_with_MLP Learning Rate        1.000000e+00  0.000000e+00   
                 Weight Decay         0.000000e+00  1.000000e+00   
                 Average of Metrics   5.296930e-01  8.968301e-03   
rnn_with_MLP     Learning Rate        1.000000e+00  6.029375e-17   
                 Weight Decay         6.029375e-17  1.000000e+00   
                 Average of Metrics  -2.367834e-01  3.857105e-01   
simple_ANN       Learning Rate        1.000000e+00  0.000000e+00   
                 Weight Decay         0.000000e+00  1.000000e+00   
                 Average of Metrics  -7.503930e-01  3.790782e-01   

                                     Average of Metrics  
Model                                                    
encoder_with_MLP Learning Rate                 0.529693  
                 Weight Decay                  0.008968  
                 Average of Metrics            1.000000  
rnn_with_MLP     Learning Rate                -0.236783  
                 Weight Decay                  0.385711  
                 Average of Metrics            1.000000  
simple_ANN       Learning Rate                -0.750393  
                 Weight Decay                  0.379078  
                 Average of Metrics            1.000000